<a href="https://colab.research.google.com/github/jhill1/teaching_scripts/blob/main/stream_power_channels_class_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Quantifying river channel evolution with Landlab
These exercises are based on a project orginally designed by Kelin Whipple at Arizona State University. This notebook was created by Nicole Gasparini at Tulane University. Edited for University of York "Earth Processes and Landform" students by Jon Hill.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

**What is this notebook?**

This notebook illustrates the evolution of detachment-limited channels in an actively uplifting landscape. The landscape evolves according to the equation:

\begin{equation}
 \frac{d z}{d t} = -K_\text{sp} A^{m_{sp}} S^{n_{sp}} + U
\end{equation}

Here, $K_{sp}$ is the erodibility coefficient on fluvial incision, which is thought to be positively correlated with climate wetness, or storminess (this is hard to quantify) and to be negatively correlated with rock strength (again, rock strength is hard to quantify). $m_{sp}$ and $n_{sp}$ are positive exponents, usually thought to have a ratio, $m_{sp}/n_{sp} \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-\frac{dz}{dx}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation. (If slope is negative there is no fluvial erosion.) $U$ is an externally-applied rock uplift field.

The fluvial erosion term is also known as the stream power equation. Before using this notebook you should be familiar with this equation from what I've covered in the lectures.

For a great overview of the stream power equation, see: 

- Whipple and Tucker, 1999, Dynamics of the stream-power river incision model: Implications for height limits of mountain ranges, landscape response timescales, and research needs, Journal of Geophysical Research.

For some great illustrations of modelling with the stream power equation, see:

- Tucker and Whipple, 2002, Topographic outcomes predicted by stream erosion models: Sensitivity analysis and intermodel comparison, Journal of Geophysical Research.

Helpful background on landscape sensitivity to rock uplift rates and patterns can be found here:

- Kirby and Whipple, 2012, Expression of active tectonics in erosional landscapes, Journal of Structural Geology.

**What will you do and what is the point?**

The idea of this practical is to think about how a river system goes back into equilibrium following some change in the system. This is what we've been covering in the lectures. We will create a steady-state system, then perturb it to see how and how long it takes to go back into equilibrium.

In this exercise you will modify the code to alter rock uplift rates and patterns and the erodibility coefficient that controls fluvial channel form.

Start at the top by reading each block of text and sequentially running each code block (click the little "play" symbol on the left of a block of code). 

If you just change one code block and rerun only that code block, only the parts of the code in that code block will be updated. (E.g. if you change parameters but don't reset the code blocks that initialize run time or topography, then these values will not be reset.) 

**STUDENTS: Questions to answer before starting this assignment.**

Answer these questions before running the notebook.

1. What do you think will happen to total relief (defined as the maximum minus the minimum elevation, here area is fixed) and channel slope at steady state if $K_{sp}$ is uniformly increased (i.e. we make the landscape more "erodable")?
2. What do you think will happen to total relief and channel slope at steady state if $U$ is uniformly increased, like if the sea level rise rate dropped?
3. How do you think a steady-state landscape with a uniform low rock uplift rate will respond if rock uplift is non-uniformly increased e.g. via an earthquake?

# Our code starts here

First we have to buld some python modules and install them. Bit of a hack here. The Google Colab build comes with a module called tomli, which we need, but it has version 2.0.0. We need version less than 2.0.0, so we install the previous version 1.2.3. However, to get this picked up, we need to restart the run time. Hence we run os.kill, which will cause this block to fail (but it hasn't! We *made* it fail). The runtime will automatically restart after being killed.

In [ ]:
!pip install landlab
!pip install tomli==1.2.3
import os
os.kill(os.getpid(), 9)

**Now on to the code...**

First we have to import the parts of Python and Landlab that are needed to run this code. You should not have to change this first code block. First, let's check we have the right version of tomli (should be 1.2.3: if not stick your hand up!)

In [ ]:
import tomli
tomli.__version__


Now onto the code proper. This is our first code block. Note the comment which numbers the blocks and I refer to those throughout the text.

In [ ]:
# Code block 1

import copy

import numpy as np
from matplotlib import pyplot as plt

from landlab import RasterModelGrid, imshow_grid
from landlab.components import (
    ChannelProfiler,
    ChiFinder,
    FlowAccumulator,
    SteepnessFinder,
    StreamPowerEroder,
)
from landlab.io import write_esri_ascii

#below is to make plots show up in the notebook
%matplotlib inline 

Make a grid and set boundary conditions. 

In [ ]:
# Code Block 2

number_of_rows = 50 # number of raster cells in vertical direction (y)
number_of_columns = 100 # number of raster cells in horizontal direction (x)
dxy  = 200 # side length of a raster model cell, or resolution [m]

# Below is a raster (square cells) grid, with equal width and height 
mg1 = RasterModelGrid((number_of_rows,number_of_columns), dxy)

# Set boundary conditions - only the south side of the grid is open.
# Boolean parameters are sent to function in order of
# east, north, west, south.
mg1.set_closed_boundaries_at_grid_edges(True, True, True, False)

Here we make the initial grid of elevation of zeros with a very small amount of noise to make a more pleasing network.

In [ ]:
# Code Block 3

np.random.seed(87) # seed set so our figures are reproducible
mg1_noise = np.random.rand(mg1.number_of_nodes)/1000. # intial noise on elevation gri

# set up the elevation on the grid
z1 = mg1.add_zeros('node', 'topographic__elevation')
z1 += mg1_noise

Set parameters related to time.

In [ ]:
# Code Block 4

tmax = 5E5  # time for the model to run [yr] (Original value was 5E5 yr for reference when you edit this)
dt = 1000.0 # time step [yr] (Original value was 1000 yr)
total_time = 0 # amount of time the landscape has evolved [yr]
# total_time will increase as you keep running the code.

t = np.arange(0, tmax, dt) # each of the time steps that the code will run

Set parameters for incision and intializing all of the process components that do the work. We also initialize tools for quantifying the landscape.

In [ ]:
# Code Block 5

# Original K_sp value is 1e-5
K_sp = 1.E-5  # units vary depending on m_sp and n_sp 
m_sp = 0.5 # exponent on drainage area in stream power equation
n_sp = 1.0 # exponent on slope in stream power equation

frr = FlowAccumulator(mg1) # intializing flow routing
spr = StreamPowerEroder(mg1, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp, threshold_sp=0) #initializing stream power incision

theta = m_sp/n_sp 
# initialize the component that will calculate channel steepness
sf = SteepnessFinder(mg1, reference_concavity=theta, min_drainage_area=1000.)
# initialize the component that will calculate the chi index
cf = ChiFinder(mg1, min_drainage_area=1000., reference_concavity=theta, use_true_dx=True)

Initialize rock uplift rate. This will need to be changed later.

In [ ]:
# Code Block 6

#  uplift_rate [m/yr] (Original value is 0.0001 m/yr)
uplift_rate = np.ones(mg1.number_of_nodes)*0.0001 

Now for the code loop. 

Note that you can rerun Code Block 7 many times, and as long as you don't reset the elevation field (Code Block 3), it will take the already evolved landscape and evolve it even more. If you want to change parameters in other code blocks (e.g. Code Block 5 or 6), you can do that too, and as long as you don't reset the elevation field (Code Block 3) the new parameters will apply on the already evolved topography. 

In [ ]:
# Code Block 7

for ti in t:
    z1[mg1.core_nodes] += uplift_rate[mg1.core_nodes]*dt # uplift the landscape
    frr.run_one_step() # route flow
    spr.run_one_step(dt) # fluvial incision
    total_time += dt # update time keeper
    print(total_time)

Plot the topography.

In [ ]:
# Code Block 8

imshow_grid(mg1, 'topographic__elevation', grid_units=('m', 'm'),
                var_name='Elevation (m)')
title_text = '$K_{sp}$='+str(K_sp) + '; $time$='+str(total_time) + 'yr; $dx$='+str(dxy) + 'm'
plt.title(title_text)

max_elev = np.max(z1)
print('Maximum elevation is ', np.max(z1))

Plot the slope and area data at each point on the landscape (in log-log space).

In [ ]:
# Code Block 9

indices = np.where(mg1.status_at_node[mg1.at_node['flow__receiver_node']] == 0)
plt.loglog(mg1.at_node['drainage_area'][indices],
           mg1.at_node['topographic__steepest_slope'][indices], 'b.')
plt.ylabel('Topographic slope')
plt.xlabel('Drainage area (m^2)')
title_text = '$K_{sp}$='+str(K_sp) + '; $time$='+str(total_time) + 'yr; $dx$='+str(dxy) + 'm'
plt.title(title_text)

It is slightly easier to interpret slope-area data when we look at a single channel, rather than the entire landscape. Below we plot the profile and slope-area data for the three largest channels on the landscape.

In [ ]:
# Code Block 10

# profile the largest channels, set initially to find the mainstem channel in the three biggest watersheds
# you can change the number of watersheds, or choose to plot all the channel segments in the watershed that
# have drainage area below the threshold (here we have set the threshold to the area of a grid cell).
prf = ChannelProfiler(mg1,
                      number_of_watersheds=3,
                      main_channel_only=True,
                      minimum_channel_threshold=dxy**2)
prf.run_one_step()

# plot the elevation as a function of distance upstream
plt.figure(1)
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m"
prf.plot_profiles(xlabel='distance upstream (m)',
                  ylabel='elevation (m)',
                  title=title_text)

# plot the location of the channels in map view
plt.figure(2)
prf.plot_profiles_in_map_view()

# slope-area data in just the profiled channels
plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(
            mg1.at_node["drainage_area"][profile_ids],
            mg1.at_node["topographic__steepest_slope"][profile_ids],
            '.',
            color=color,
            label=label,
        )

plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m"
plt.title(title_text)

The channel steepness index is a useful index to quantify fluvial channels. Below we plot the steepness index in the same three largest channels, and also plot steepness index across the grid.

In [ ]:
# Code Block 11


# calculate channel steepness
sf.calculate_steepnesses()

# plots of steepnes vs. distance upstream in the profiled channels
plt.figure(4)

for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(
            distance_upstream,
            mg1.at_node["channel__steepness_index"][profile_ids],
            'x',
            color=color,
            label=label,
        )

plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
plt.title(
    f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
)

# channel steepness map
plt.figure(15)
imshow_grid(
    mg1,
    "channel__steepness_index",
    grid_units=("m", "m"),
    var_name="Steepness index ",
    cmap="jet",
)
plt.title(
    f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
)

If you have a grid that you want to export, uncomment and edit the appropriate lines below and run the code block.

In [ ]:
# Code Block 12

## Below has the name of the file that data will be written to.
## You need to change the name of the file every time that you want
## to write data, otherwise you will get an error.
## This will write to the directory that you are running the code in.
write_file_name = 'data_file.txt'
## Below is writing elevation data in the ESRI ascii format so that it can
## easily be read into Arc GIS or back into Landlab.
from landlab.io import write_esri_ascii
write_esri_ascii(write_file_name, mg1, 'topographic__elevation')
# Click on the folder icon on the left and you'll see data_file.txt. Hover over and press the three-dot menu to download
# Edit the filename if you want to write out different files

After running every code block once, has the landscape reached steady state? Answer: NO! How do you know? After you think about this, you are ready to complete this project.

You might find it useful to create a Word or Google doc to copy and paste blocks of code, figures and type your own notes. You can use any or all of these different figures to help you with the questions below. (Download or screenshoot the figures.) 

First job is to get a steady state system. 

1. **Steady state with low uplift rate. ** Using the initial parameters provided, run the landscape to steady state. Do this by running the main evolution loop - Code Block 7 - again and the different plotting blocks, until the plots show a steady state system. *How many runs did you do? How long is this in time?* You may also want to change $tmax$ in Code Block 4 instead (but might need some guess work to get the right number).

How did you know that the landscape reached steady state? 

Note the approximate time that it took to reach steady state for your own reference. (This will be useful for later questions.) Include appropriate plots in your notes. (If you want to analyze these landscapes outside of Landlab or save for later, make sure you save the elevation data to a text file (Code Block 12).

**NOTE: For the rest of the questions you should use Code Blocks 13-19. These will allow you to use the steady-state landscape created for question 1 - referred to here as the 'base landscape' - as the initial condition. Run Code block 13, which creates copies of that initial landscape. Then start by editing what you need to in Code Blocks 14 - 16. Run these each once, sequentially. You can run Code Block 16, the time loop, as many times as you need to, along with Code Blocks 17-19, which produce plots.**

For each question, re-run code block 13 to get back to that initial landscape. If it all goes wrong, start from code block 2, but you can edit the $tmax$ parameter to get to steady state in one click...

2. **Transient landscape responding to an increase in rock uplift.** Use the base landscape and increase rock uplift uniformly by a factor of 4 to 0.0004 m/yr. Make sure you update the rock uplift rate (Code Block 16) and ensure that $tmax$ is 1e5 yrs and $dt$ is 500 yrs (Code Block 15). Run this until the maximum elevation in the grid is ~ 250 m and observe how the landscape gets to this elevation, i.e. plot intermediate steps. What patterns do you see in the supporting plots that illustrate this type of transient? What aspects of the landscape have changed in comparison with the base landscape from question 1?

3. **Increase erodibility.** Start again from the base landscape (i.e. re-run code block 13), but this time increase $K_{sp}$ to 2E-5 (Code Block 13). Make sure rock uplift rate is set to the original value of 0.0001 m/yr (Code Block 15). Set $tmax$ to 1e5 yrs (Code Block 14). Run for 1e5 yrs and save the plots that you think are diagnostic. Run for another 1e5 yrs and save plots again. Now run for 5e5 yrs and save plots again. Quantitatively describe how the landscape evolves in response to the increase in erodibility and provide supporting plots. What could cause a uniform increase in erodibility?

4. **Spatially varible uplift - discrete, massive earthquake.** Start again from the base landscape  (Code Block 13), and make sure that $K_{sp}$ = 1E-5. Now add a seismic event to this steady state landscape - a fault that runs horizontally across the landscape at y = 4000 m, and instantaneously uplifts half the landscape by 10 meters (Code Block 15 - uncomment the five lines in the first *IMPORTANT* comment block). In this case, we will keep background uplift uniform at 0.0001 m/yr. Set $tmax$ to 1e5 yrs and $dt$ to 500 yrs (Code Block 15) before evolving the landscape after the fault. Now run the time loop four times and look at the different plots after each loop. How does the landscape respond to this fault? What patterns do you see in the supporting plots that illustrate this type of transient? Which patterns, if any, are diagnostic of a channel response to an earthquake? 

6. **Final Reflection.** Was your initial insight into how parameters would effect the landscape correct?

Feel free to play abound some more. How might uplift rate increase and erodibility change play out if they happen at the same time? How might we differentiate between these things happening?

In [ ]:
# Code Block 13

number_of_rows = 50  # number of raster cells in vertical direction (y)
number_of_columns = 100  # number of raster cells in horizontal direction (x)
dxy2 = 200  # side length of a raster model cell, or resolution [m]

# Below is a raster (square cells) grid, with equal width and height
mg2 = RasterModelGrid((number_of_rows, number_of_columns), dxy2)

# Set boundary conditions - only the south side of the grid is open.
# Boolean parameters are sent to function in order of
# east, north, west, south.
mg2.set_closed_boundaries_at_grid_edges(True, True, True, False)

z2 = copy.copy(z1)  # initialize the elevations with the steady state
# topography produced for question 1
z2 = mg2.add_field("topographic__elevation", z2, at="node")

# K_sp value for base landscape is 1e-5
K_sp2 = 1e-5  # units vary depending on m_sp and n_sp
m_sp2 = 0.5  # exponent on drainage area in stream power equation
n_sp2 = 1.0  # exponent on slope in stream power equation

frr2 = FlowAccumulator(mg2)  # intializing flow routing
spr2 = StreamPowerEroder(
    mg2, K_sp=K_sp2, m_sp=m_sp2, n_sp=n_sp2,
    threshold_sp=0.0)  # initializing stream power incision

theta2 = m_sp2 / n_sp2
# initialize the component that will calculate channel steepness
sf2 = SteepnessFinder(mg2,
                      reference_concavity=theta2,
                      min_drainage_area=1000.0)
# initialize the component that will calculate the chi index
cf2 = ChiFinder(mg2,
                min_drainage_area=1000.0,
                reference_concavity=theta2,
                use_true_dx=True)

In [ ]:
# Code Block 14

tmax = 1E5  # time for the model to run [yr] (Original value was 5E5 yr)
dt = 500.0 # time step [yr] (Original value was 500 yr)
total_time = 0 # amount of time the landscape has evolved [yr]
# total_time will increase as you keep running the code.

t = np.arange(0, tmax, dt) # each of the time steps that the code will run

In [ ]:
# Code Block 15

#  uplift_rate [m/yr] (value was 0.0001 m/yr for base landscape)
uplift_rate = np.ones(mg2.number_of_nodes)*0.0001

## If you want to add a one-time event that uplifts only part of the 
## landscape, uncomment the 3 lines below
#fault_location = 4000  # [m]
#uplift_amount = 10 # [m]
#z2[np.where(mg2.node_y>fault_location)] += uplift_amount 

## IMPORTANT! To use the below fault generator, comment the one-time 
## uplift event above if it isn't already commented out.
## Code below creates a fault horizontally across the grid.
## Uplift rates are greater where y values  > fault location.
## To use, uncomment the 5 code lines below and edit to your values
#fault_location = 4000  # [m]
#low_uplift_rate = 0.0001 # [m/yr]
#high_uplift_rate = 0.0004 # [m/yr]
#uplift_rate[np.where(mg2.node_y<fault_location)] = low_uplift_rate
#uplift_rate[np.where(mg2.node_y>fault_location)] = high_uplift_rate

## IMPORTANT! To use below rock uplift gradient, comment the two
## uplift options above if they aren't already commented out.
## If you want a linear gradient in uplift rate 
## (increasing uplift into the range),
## uncomment the 4 code lines below and edit to your values.
#low_uplift_rate = 0.0001 # [m/yr]
#high_uplift_rate = 0.0004 # [m/yr]
## below is uplift gradient per node row index, NOT row value in meters
#uplift_rate_gradient = (high_uplift_rate - low_uplift_rate)/(number_of_rows-3)
#uplift_rate = low_uplift_rate + ((mg2.node_y / dxy)-1) * uplift_rate_gradient

In [ ]:
# Code Block 16

for ti in t:
    z2[mg1.core_nodes] += uplift_rate[mg2.core_nodes]*dt # uplift the landscape
    frr2.run_one_step() # route flow
    spr2.run_one_step(dt) # fluvial incision
    total_time += dt # update time keeper
    print(total_time)

In [ ]:
# Code Block 17
# Plot topography

plt.figure(7)
imshow_grid(mg2, 'topographic__elevation', grid_units=('m', 'm'),
                var_name='Elevation (m)')
title_text = '$K_{sp}$='+str(K_sp2) + '; $time$='+str(total_time) + 'yr; $dx$='+str(dxy) + 'm'
plt.title(title_text)

max_elev = np.max(z2)
print('Maximum elevation is ', np.max(z2))

In [ ]:
# Code Block 18
# Plot Channel Profiles and slope-area data along the channels


prf2 = ChannelProfiler(mg2,
                       number_of_watersheds=3,
                       main_channel_only=True,
                       minimum_channel_threshold=dxy**2)
prf2.run_one_step()

# plot the elevation as a function of distance upstream
plt.figure(9)
title_text = f"$K_{{sp}}$={K_sp2}; $time$={total_time} yr; $dx$={dxy} m"
prf2.plot_profiles(xlabel='distance upstream (m)',
                  ylabel='elevation (m)',
                  title=title_text)

# plot the location of the channels in map view
plt.figure(10)
prf2.plot_profiles_in_map_view()

# slope-area data in just the profiled channels
plt.figure(11)
for i, outlet_id in enumerate(prf2.data_structure):
    for j, segment_id in enumerate(prf2.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf2.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(
            mg2.at_node["drainage_area"][profile_ids],
            mg2.at_node["topographic__steepest_slope"][profile_ids],
            '.',
            color=color,
            label=label,
        )

plt.legend(loc="lower left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = f"$K_{{sp}}$={K_sp2}; $time$={total_time} yr; $dx$={dxy2} m"
plt.title(title_text)

In [ ]:
# Code Block 19
# Plot channel steepness along profiles and across the landscape

# calculate channel steepness
sf2.calculate_steepnesses()

# plots of steepnes vs. distance upstream in the profiled channels
plt.figure(14)

for i, outlet_id in enumerate(prf2.data_structure):
    for j, segment_id in enumerate(prf2.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf2.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(
            distance_upstream,
            mg2.at_node["channel__steepness_index"][profile_ids],
            'x',
            color=color,
            label=label,
        )

plt.xlabel("distance upstream (m)")
plt.ylabel("steepness index")
plt.legend(loc="upper left")
plt.title(
    f"$K_{{sp}}$={K_sp2}; $time$={total_time} yr; $dx$={dxy2} m; concavity={theta2}"
)

# channel steepness map
plt.figure(15)
imshow_grid(
    mg2,
    "channel__steepness_index",
    grid_units=("m", "m"),
    var_name="Steepness index ",
    cmap="jet",
)
plt.title(
    f"$K_{{sp}}$={K_sp2}; $time$={total_time} yr; $dx$={dxy2} m; concavity={theta2}"
)